In [1]:
%matplotlib inline
from detail import Detail
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import os,sys
import pylab
import json
import random
from detail import mask as maskUtil
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

In [2]:
def shutup(func):
    def wrapper(*args, **kwargs):        
        save_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
        res = func(*args, **kwargs)
        sys.stdout = save_stdout
        return res
    return wrapper

In [3]:
annFile='/Users/zhishuaizhang/pascal_in_detail_data/trainval_merged.json' # annotations
imgDir='/Users/zhishuaizhang/pascal_in_detail/VOCdevkit/VOC2010/JPEGImages' # jpeg images
phase='trainval'

In [4]:
# initialize detail api for instance annotations
details = Detail(annFile, imgDir, phase)

print('\n***info***')
details.info()

loading annotations into memory...
Done (t=8.42s)
creating index...
index created! 8.13s

***info***
description: This is the 3.0 version of the CVPR 2017 PASCAL in Detail dataset.
version: 3.0
year: 2017
contributor: https://sites.google.com/view/pasd
date_created: 2017-07-18


In [5]:
from detail import instsegEval
instsegeval = instsegEval.instsegEval(details)

In [27]:
# Test it on a toy dataset
instsegeval.params.imgIds = [2008000512, 2008000514]
instsegeval.params.catIds = [25, 360, 420, 284, 284, 284, 284, 284, 33, 232, 349, 440]
# Create a detection result from groundtruth of 500 images
# This cell will take 10 mins in my computer
gts = [(i,j,maskUtil.encode(np.asfortranarray((details.getMask(i,cat=j).astype(np.uint8)==k).astype(np.uint8)))) \
    for i in instsegeval.params.imgIds \
        for j in [_['category_id'] for _ in details.getCats(imgs=i)] \
            for k in (set(np.unique(details.getMask(i,cat=j).astype(np.uint8)))-{0})]
all_gt = []
imgIds_ = []
catIds_ = []
[all_gt.append({'image_id':_[0],'category_id':_[1],'segmentation':{"size":_[2]['size'],"counts":_[2]['counts'].decode("utf-8")},'score':100.}) or \
    imgIds_.append(_[0]) or catIds_.append(_[1]) for _ in gts]
with open('/Users/zhishuaizhang/github_repo/detail-api/res/inst_seg_toy_one_wall_miss.json', 'w') as outfile:
    json.dump(all_gt, outfile)

In [30]:
# Evaluate the toy_corrent dataset
import time
tic=time.time()
instsegeval.loadSegRes('/Users/zhishuaizhang/github_repo/detail-api/res/inst_seg_toy_correct.json')
instsegeval.evaluate()
instsegeval.accumulate()
toc=time.time()
print(toc-tic)

2.769188165664673


In [31]:
print(instsegeval.ap[9,:,-1]) # AP for IOU>0.95, and unlimited detections
print(instsegeval.ap[5,:,-1]) # AP for IOU>0.75, and unlimited detections
print(instsegeval.ap[0,:,-1]) # AP for IOU>0.5, and unlimited detections
## Work as expected.

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [51]:
# Evaluate the toy_one_wall_miss dataset (There is only one wall instance)
import time
tic=time.time()
instsegeval.loadSegRes('/Users/zhishuaizhang/github_repo/detail-api/res/inst_seg_toy_one_wall_miss.json')
instsegeval.evaluate()
instsegeval.accumulate()
toc=time.time()
print(toc-tic)

3.058359146118164


In [52]:
print(instsegeval.ap[9,:,-1]) # AP for IOU>0.95, and unlimited detections
print(instsegeval.ap[5,:,-1]) # AP for IOU>0.75, and unlimited detections
print(instsegeval.ap[0,:,-1]) # AP for IOU>0.5, and unlimited detections
## Work as expected.

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.]


In [6]:
# Create a detection result from groundtruth of 500 images
# This cell will take 10 mins in my computer
gts = [(i,j,maskUtil.encode(np.asfortranarray((details.getMask(i,cat=j).astype(np.uint8)==k).astype(np.uint8)))) \
    for i in list(details.imgs.keys())[:500] \
        for j in [_['category_id'] for _ in details.getCats(imgs=i)] \
            for k in (set(np.unique(details.getMask(i,cat=j).astype(np.uint8)))-{0})]
instsegeval = instsegEval.instsegEval(details)
all_gt = []
imgIds_ = []
catIds_ = []
[all_gt.append({'image_id':_[0],'category_id':_[1],'segmentation':{"size":_[2]['size'],"counts":_[2]['counts'].decode("utf-8")},'score':100.}) or \
    imgIds_.append(_[0]) or catIds_.append(_[1]) for _ in gts]
with open('/Users/zhishuaizhang/github_repo/detail-api/res/inst_seg_correct.json', 'w') as outfile:
    json.dump(all_gt, outfile)

In [7]:
# Simplify the image and category list to make debug / test easier
instsegeval.params.imgIds = list(set(imgIds_))
instsegeval.params.catIds = list(set(catIds_))

In [17]:
# Evaluate those 500 images
# This cell will take 10 mins in my computer
import time
tic=time.time()
instsegeval.loadSegRes('/Users/zhishuaizhang/github_repo/detail-api/res/inst_seg_correct.json')
instsegeval.evaluate()
instsegeval.accumulate()
toc=time.time()
print(toc-tic)

In [18]:
print(instsegeval.ap[9,:,-1]) # AP for IOU>0.95, and unlimited detections
print(instsegeval.ap[5,:,-1]) # AP for IOU>0.75, and unlimited detections
print(instsegeval.ap[0,:,-1]) # AP for IOU>0.5, and unlimited detections
## Work as expected.

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.]
